In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nf-unsw-nb15-v2csv/NF-UNSW-NB15-v2.csv


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('/kaggle/input/nf-unsw-nb15-v2csv/NF-UNSW-NB15-v2.csv')
df.head()

,IPV4_SRC_ADDR,L4_SRC_PORT,IPV4_DST_ADDR,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label,Attack
0,59.166.0.5,1305,149.171.126.8,21,6,1.0,9,1,193,3,...,0,7240,0,0,0,0,0,331.0,0,Benign
1,59.166.0.5,1305,149.171.126.8,21,6,1.0,261,5,469,7,...,8688,8688,18944,74,0,0,0,230.0,0,Benign
2,59.166.0.5,1305,149.171.126.8,21,6,1.0,481,9,750,11,...,10136,10136,33792,132,0,0,0,229.0,0,Benign
3,59.166.0.5,1305,149.171.126.8,21,6,1.0,701,13,1054,15,...,11584,11584,48640,190,0,0,0,125.0,0,Benign
4,59.166.0.5,1305,149.171.126.8,21,6,1.0,1031,19,1474,21,...,14480,13032,64256,251,0,0,0,230.0,0,Benign


In [4]:
df.columns

Index(['IPV4_SRC_ADDR', 'L4_SRC_PORT', 'IPV4_DST_ADDR', 'L4_DST_PORT',
       'PROTOCOL', 'L7_PROTO', 'IN_BYTES', 'IN_PKTS', 'OUT_BYTES', 'OUT_PKTS',
       'TCP_FLAGS', 'CLIENT_TCP_FLAGS', 'SERVER_TCP_FLAGS',
       'FLOW_DURATION_MILLISECONDS', 'DURATION_IN', 'DURATION_OUT', 'MIN_TTL',
       'MAX_TTL', 'LONGEST_FLOW_PKT', 'SHORTEST_FLOW_PKT', 'MIN_IP_PKT_LEN',
       'MAX_IP_PKT_LEN', 'SRC_TO_DST_SECOND_BYTES', 'DST_TO_SRC_SECOND_BYTES',
       'RETRANSMITTED_IN_BYTES', 'RETRANSMITTED_IN_PKTS',
       'RETRANSMITTED_OUT_BYTES', 'RETRANSMITTED_OUT_PKTS',
       'SRC_TO_DST_AVG_THROUGHPUT', 'DST_TO_SRC_AVG_THROUGHPUT',
       'NUM_PKTS_UP_TO_128_BYTES', 'NUM_PKTS_128_TO_256_BYTES',
       'NUM_PKTS_256_TO_512_BYTES', 'NUM_PKTS_512_TO_1024_BYTES',
       'NUM_PKTS_1024_TO_1514_BYTES', 'TCP_WIN_MAX_IN', 'TCP_WIN_MAX_OUT',
       'ICMP_TYPE', 'ICMP_IPV4_TYPE', 'DNS_QUERY_ID', 'DNS_QUERY_TYPE',
       'DNS_TTL_ANSWER', 'FTP_COMMAND_RET_CODE', 'Label', 'Attack'],
      dtype='object')

In [5]:
df['Label'].unique()

array([0, 1])

In [6]:
df['Attack'].unique()

array(['Benign', 'Exploits', 'Generic', 'Fuzzers', 'Backdoor', 'DoS',
       'Reconnaissance', 'Shellcode', 'Worms', 'Analysis'], dtype=object)

In [7]:
df.shape

(2390275, 45)

In [8]:
label_counts = df['Label'].value_counts()
label_counts

Label
0    2295222
1      95053
Name: count, dtype: int64

In [11]:
total_rows = len(df)

specific_label = 1
specific_label_count = label_counts[specific_label]
specific_label_percentage = (specific_label_count / total_rows) * 100

specific_label_percentage

3.9766554057587515

In [12]:
df = df.drop(columns=['Attack', 'Label'])

In [13]:
df.columns = df.columns.str.lower()

In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import joblib

In [18]:
df.head()

,ipv4_src_addr,l4_src_port,ipv4_dst_addr,l4_dst_port,protocol,l7_proto,in_bytes,in_pkts,out_bytes,out_pkts,...,num_pkts_512_to_1024_bytes,num_pkts_1024_to_1514_bytes,tcp_win_max_in,tcp_win_max_out,icmp_type,icmp_ipv4_type,dns_query_id,dns_query_type,dns_ttl_answer,ftp_command_ret_code
0,59.166.0.5,1305,149.171.126.8,21,6,1.0,9,1,193,3,...,0,0,0,7240,0,0,0,0,0,331.0
1,59.166.0.5,1305,149.171.126.8,21,6,1.0,261,5,469,7,...,0,0,8688,8688,18944,74,0,0,0,230.0
2,59.166.0.5,1305,149.171.126.8,21,6,1.0,481,9,750,11,...,0,0,10136,10136,33792,132,0,0,0,229.0
3,59.166.0.5,1305,149.171.126.8,21,6,1.0,701,13,1054,15,...,0,0,11584,11584,48640,190,0,0,0,125.0
4,59.166.0.5,1305,149.171.126.8,21,6,1.0,1031,19,1474,21,...,0,0,14480,13032,64256,251,0,0,0,230.0


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2390275 entries, 0 to 2390274
Data columns (total 43 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   ipv4_src_addr                object 
 1   l4_src_port                  int64  
 2   ipv4_dst_addr                object 
 3   l4_dst_port                  int64  
 4   protocol                     int64  
 5   l7_proto                     float64
 6   in_bytes                     int64  
 7   in_pkts                      int64  
 8   out_bytes                    int64  
 9   out_pkts                     int64  
 10  tcp_flags                    int64  
 11  client_tcp_flags             int64  
 12  server_tcp_flags             int64  
 13  flow_duration_milliseconds   int64  
 14  duration_in                  int64  
 15  duration_out                 int64  
 16  min_ttl                      int64  
 17  max_ttl                      int64  
 18  longest_flow_pkt             int64  
 19  

In [19]:
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df.drop(columns=['ipv4_src_addr', 'ipv4_dst_addr']))

In [20]:
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [21]:
model = IsolationForest(contamination=0.04)

In [22]:
model.fit(df_scaled)

IsolationForest(contamination=0.04)

In [23]:
joblib.dump(model, 'isolation_forest_model.pkl')

['isolation_forest_model.pkl']

In [25]:
predictions = model.predict(df_scaled)
predictions = [1 if x == -1 else 0 for x in predictions]

In [28]:
df['predictions'] = predictions

In [29]:
df['predictions'].unique()

array([0, 1])

In [30]:
df['predictions'].value_counts()

predictions
0    2294692
1      95583
Name: count, dtype: int64